In [1]:
import base64
import collections
import datetime
import itertools
import json
import os
import pickle
import random
import re
import sys
import time

import matplotlib as mpl
import matplotlib.pyplot as plt
#import nltk
import numpy as np
import pandas as pd
#import PIL
import pylab
import scipy
import seaborn as sns
import sklearn
from sklearn import *
import statsmodels as sm
import xgboost as xgb

np.random.seed(1337)

%matplotlib inline

sns.set(font_scale=1.0)
mpl.rcParams['figure.figsize'] = 12, 8
sns.set_style('whitegrid')
sns.set_palette(sns.color_palette('muted'))

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/usr/local/lib/python2.7/

# Load the Data

In [51]:
tr_all = pd.read_csv('tmp/combined_train_selected_all.csv', index_col='id').sample(frac=1.0)
trl_all = tr_all.interest_level.copy()
trl_all_d = pd.get_dummies(tr_all.interest_level)[['low', 'medium', 'high']]
ilm = {
        'low': 0,
        'medium': 1,
        'high': 2}
trl_all_n = tr_all.interest_level.map(ilm)
del tr_all['interest_level']
te_all = pd.read_csv('tmp/combined_test_selected_all.csv', index_col='id')
del te_all['interest_level']

# Split

In [52]:
tt_lim = int(0.8 * len(tr_all))
ho_tr_all_f = tr_all[:tt_lim]
ho_te_all_f = tr_all[tt_lim:]
ho_tr_all_l = trl_all[:tt_lim]
ho_tr_all_l_d = trl_all_d[:tt_lim]
ho_tr_all_l_n = trl_all_n[:tt_lim]
ho_te_all_l = trl_all[tt_lim:]
ho_te_all_l_d = trl_all_d[tt_lim:]
ho_te_all_l_n = trl_all_n[tt_lim:]

# Logistic Regression

In [71]:
preds = {}
preds_p = {}
for t in ['low', 'medium', 'high']:
    print t
    m = sklearn.linear_model.LogisticRegression()
    m.fit(ho_tr_all_f.fillna(0), ho_tr_all_l_d[t])
    preds[t] = m.predict(ho_te_all_f.fillna(0))
    preds_p[t] = m.predict_proba(ho_te_all_f.fillna(0))[:, 1]
    print sklearn.metrics.log_loss(ho_te_all_l_d[t], preds_p[t])
    print sklearn.metrics.confusion_matrix(ho_te_all_l_d[t], preds[t])
    print '=' * 30
print sklearn.metrics.log_loss(
    ho_te_all_l_n,
    pd.DataFrame({t:preds_p[t] for t in ['low', 'medium', 'high']})[['low', 'medium', 'high']])

low
0.522342075738
[[ 847 2163]
 [ 493 6368]]
medium
0.491995892049
[[7534   69]
 [2219   49]]
high
0.226819385201
[[9106   23]
 [ 702   40]]
0.683355238906
